In [9]:
import pandas as pd
import matplotlib
matplotlib.use('Agg')
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta
import os

# display all columns and rows
pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
TRAINING_WINDOW = 14
BLOCK_WINDOW = 1

users = [
'u004_rct@eureka',
'u005_rct@eureka',
'u006_rct@eureka',
'u007_rct@eureka',
'u008_rct@eureka',
'u009_rct@eureka',
'u010_rct@eureka',
'u011_rct@eureka',
'u012_rct@eureka',
'u013_rct@eureka',
'u014_rct@eureka',
'u015_rct@eureka',
'u016_rct@eureka',
'u017_rct@eureka',
'u018_rct@eureka',
'u019_rct@eureka',
'u020_rct@eureka',
'u021_rct@eureka',
'u022_rct@eureka',
'u023_rct@eureka',
'u024_rct@eureka',
'u025_rct@eureka',
'u026_rct@eureka',
'u027_rct@eureka',
'u028_rct@eureka',
'u029_rct@eureka',
'u030_rct@eureka',
'u031_rct@eureka',
'u032_rct@eureka',
'u033_rct@eureka',
'u034_rct@eureka',
'u035_rct@eureka',
'u036_rct@eureka',
'u037_rct@eureka',
'u038_rct@eureka',
'u039_rct@eureka',
'u040_rct@eureka',
'u041_rct@eureka',
'u042_rct@eureka',
'u043_rct@eureka',
'u044_rct@eureka',
'u045_rct@eureka',
'u046_rct@eureka',
'u047_rct@eureka',
'u048_rct@eureka',
'u049_rct@eureka',
'u050_rct@eureka',
'u051_rct@eureka',
'u052_rct@eureka',
'u053_rct@eureka',
'u054_rct@eureka',
'u055_rct@eureka',
'u056_rct@eureka',
'u057_rct@eureka',
'u058_rct@eureka',
'u059_rct@eureka',
'u060_rct@eureka',
'u061_rct@eureka',
'u062_rct@eureka',
'u063_rct@eureka',
'u064_rct@eureka',
'u065_rct@eureka',
'u066_rct@eureka',
'u067_rct@eureka',
'u068_rct@eureka',
'u069_rct@eureka',
'u070_rct@eureka',
'u071_rct@eureka',
'u072_rct@eureka',
'u073_rct@eureka',
]
print('total number of users:', len(users))

def get_table(csv_file):    
    csv_file = csv_file.drop(['Unnamed: 0'], axis=1)
    csv_file['time'] = pd.to_datetime(csv_file['time'])
    return csv_file

def get_distance(ts1, ts2, mode):
    diff = ts1 - ts2        
    diff = diff.abs()    
    if 'mean' in mode:
        return diff.mean()
    elif 'median' in mode or 'med' in mode:
        return diff.median()
    elif 'max' in mode:
        return diff.max()
    else:
        print('invalid mode: 0: mean; 1: median; 2: max')
        return None

total number of users: 70


In [3]:
# get all processed feature tables
print(datetime.datetime.now())
calllog = get_table(pd.read_csv('table_calllog.csv'))
smslog = get_table(pd.read_csv('table_sms.csv'))
unlock = get_table(pd.read_csv('table_raw_unlock.csv'))
conver = get_table(pd.read_csv('table_conver.csv'))
sleep = get_table(pd.read_csv('table_sleep.csv'))
act = get_table(pd.read_csv('table_act.csv'))
light = get_table(pd.read_csv('table_light.csv'))
audio_amp = get_table(pd.read_csv('table_audio_amp.csv'))
audio_infer = get_table(pd.read_csv('table_audio_infer_new.csv'))
print(datetime.datetime.now())

ema = pd.read_csv('full_dataset.csv')
ema = ema[['study_id','eureka_id','day','ema_CALM','ema_DEPRESSED','ema_HARM','ema_HOPEFUL','ema_SEEING_THINGS','ema_SLEEPING','ema_SOCIAL','ema_STRESSED','ema_THINK','ema_VOICES','ema_neg_score','ema_pos_score','ema_resp_time_median']]

2020-05-21 16:55:28.340371
2020-05-21 17:00:24.554868


In [6]:
days_string = []
for day in ema['day']:
    days_string.append(str(day))

In [8]:
ema_days = []
for day in days_string:
    ema_days.append(pd.to_datetime(day))
ema['day'] = ema_days
ema['ema_score'] = ema['ema_neg_score'] - ema['ema_pos_score']

In [10]:
ema = ema.dropna()

In [14]:
quality_df = pd.read_csv('full_dataset.csv')
quality_df = quality_df[['eureka_id','day','quality_activity','quality_audio','quality_light']]
quality_days_string = []
for day in quality_df['day']:
    quality_days_string.append(str(day))
quality_days = []
for day in quality_days_string:
    quality_days.append(pd.to_datetime(day))
quality_df['day'] = quality_days

,eureka_id,day,quality_activity,quality_audio,quality_light
0,u004,2015-01-22,0,0,0
1,u004,2015-01-23,0,0,0
2,u004,2015-01-24,0,0,0
3,u004,2015-01-25,0,0,0
4,u004,2015-01-26,0,0,0


# compute features for short periods

In [18]:
def compute_stability_index(behavior, behavior_name, NUM_DAYS_USED, NUM_DAYS_BLOCKED, FILTERING):
    table_all_behav = pd.DataFrame(columns=['eureka_id', 'day'])
    for user in users:
#         print('current user: ' + user)
        ema_user = ema.loc[ema.eureka_id == user[0:4]]
#         ema_user = ema_user.dropna()
        table_user = ema_user.drop(['study_id'], axis=1)
        behavior_user = behavior.loc[behavior.uid == user]
        behavior_user = behavior_user.set_index('time')
        behavior_user = behavior_user.sort_index()
        behavior_user = behavior_user.reset_index()  
        
        # set of features       
        normal_cumu_med_mean_diff = []
        absolute_cumu_med_mean_diff = []
        raw_med_mean_diff = []
        
        behavior_mean = []
        behavior_std = []
        num_good_days = []
        
        # pre processing normal, absolute, raw
        behavior_user_cumu = behavior_user[['time', behavior_name + '_cumu']]
        behavior_user_cumu_normal = behavior_user[['time', behavior_name + '_cumu_normal']]
        behavior_user_raw = behavior_user[['time', behavior_name]]
        
        behavior_user_cumu = behavior_user_cumu.set_index('time')
        behavior_user_cumu_normal = behavior_user_cumu_normal.set_index('time')
        behavior_user_raw = behavior_user_raw.set_index('time')

        # get the list of available days in this behavior data for this user
        user_days = []
        user_days_filtered = []
        for date in behavior_user_cumu.index:
            timestampStr = date.strftime("%Y-%m-%d")
            if timestampStr not in user_days:
                user_days.append(timestampStr)
        for u_day in range(len(user_days)):
            user_days[u_day] = pd.to_datetime(user_days[u_day])
        
        # remove all bad days for activity, audio, light
        if 'act' in behavior_name or 'light_mean' in behavior_name or 'audio' in behavior_name:     
            quality_df_user = quality_df.loc[quality_df.eureka_id == user[0:4]]
            quality_df_user = quality_df_user.set_index('day')
            for d in user_days:
                if d in quality_df_user.index:
                    if 'act' in behavior_name:
                        if quality_df_user.loc[d]['quality_activity'] > FILTERING:
                            user_days_filtered.append(d)
                    elif 'audio' in behavior_name:
                        if quality_df_user.loc[d]['quality_audio'] > FILTERING:
                            user_days_filtered.append(d)
                    else:
                        if quality_df_user.loc[d]['quality_light'] > FILTERING:
                            user_days_filtered.append(d)
        else:
            user_days_filtered = user_days
                
        # compute feature for each ema day
        for i in range(len(table_user['day'])):
            diff_mean = []
            diff_normal_mean = []
            behavior_amount = []
            diff_raw_mean = []
            
            this_ema_day = table_user.iloc[i]['day']
#             print()
#             print('new ema day: ', this_ema_day.strftime("%Y-%m-%d"))
            starting_day = this_ema_day - timedelta(days=NUM_DAYS_USED + NUM_DAYS_BLOCKED - 1)
            if NUM_DAYS_BLOCKED > 0:
                days_unused = this_ema_day - pd.to_timedelta(np.arange(NUM_DAYS_BLOCKED), 'd')
                days_used = [u_f for u_f in user_days_filtered if u_f < this_ema_day and u_f >= starting_day]
            else:
                days_unused = []
                days_used = [u_f for u_f in user_days_filtered if u_f <= this_ema_day and u_f >= starting_day]
            for u in days_unused:
                if u in days_used:
                    days_used.remove(u)
            num_good_days.append(len(days_used))
#             print(days_used)


            # calculate and store difference values between each possible day pairs in a list    
            # Graph Data
            df_graphs = pd.DataFrame({'x': range(0,1440)})
            df_graphs_normal = pd.DataFrame({'x': range(0,1440)})
            df_graphs_raw = pd.DataFrame({'x': range(0,1440)})

            for j in range(len(days_used)):
                k = j + 1                
                while k < len(days_used):
#                     print('compute for these two days:')
#                     print(str(j) + str(k))
#                     print(days_used[j].strftime("%Y-%m-%d"))
#                     print(days_used[k].strftime("%Y-%m-%d"))
                
                    behavior_user_cumu_day1 = behavior_user_cumu[days_used[j].strftime("%Y-%m-%d")]
                    behavior_user_cumu_day2 = behavior_user_cumu[days_used[k].strftime("%Y-%m-%d")]
                    behavior_user_cumu_normal_day1 = behavior_user_cumu_normal[days_used[j].strftime("%Y-%m-%d")]
                    behavior_user_cumu_normal_day2 = behavior_user_cumu_normal[days_used[k].strftime("%Y-%m-%d")]
                    behavior_user_raw_day1 = behavior_user_raw[days_used[j].strftime("%Y-%m-%d")]
                    behavior_user_raw_day2 = behavior_user_raw[days_used[k].strftime("%Y-%m-%d")]
                    # drop time and reset the index the same
                    behavior_user_cumu_day1 = behavior_user_cumu_day1.reset_index()
                    behavior_user_cumu_day2 = behavior_user_cumu_day2.reset_index()
                    behavior_user_cumu_normal_day1 = behavior_user_cumu_normal_day1.reset_index()
                    behavior_user_cumu_normal_day2 = behavior_user_cumu_normal_day2.reset_index()
                    behavior_user_raw_day1 = behavior_user_raw_day1.reset_index()
                    behavior_user_raw_day2 = behavior_user_raw_day2.reset_index()                    
                    behavior_user_cumu_day1 = behavior_user_cumu_day1[behavior_name + '_cumu']
                    behavior_user_cumu_day2 = behavior_user_cumu_day2[behavior_name + '_cumu']
                    behavior_user_cumu_normal_day1 = behavior_user_cumu_normal_day1[behavior_name + '_cumu_normal']
                    behavior_user_cumu_normal_day2 = behavior_user_cumu_normal_day2[behavior_name + '_cumu_normal']
                    behavior_user_raw_day1 = behavior_user_raw_day1[behavior_name]
                    behavior_user_raw_day2 = behavior_user_raw_day2[behavior_name]
                    
                    diff_mean.append(get_distance(behavior_user_cumu_day1,behavior_user_cumu_day2,'mean'))
                    diff_normal_mean.append(get_distance(behavior_user_cumu_normal_day1,behavior_user_cumu_normal_day2,'mean'))
                    diff_raw_mean.append(get_distance(behavior_user_raw_day1,behavior_user_raw_day2,'mean'))
                    k = k + 1

            diff_mean = pd.Series(diff_mean)
            diff_normal_mean = pd.Series(diff_normal_mean)
            diff_raw_mean = pd.Series(diff_raw_mean)
            
            # accumulate values of each ema day for this user   
            absolute_cumu_med_mean_diff.append(diff_mean.median())
            normal_cumu_med_mean_diff.append(1.0-diff_normal_mean.median())
            raw_med_mean_diff.append(diff_raw_mean.median())

            for j in range(len(days_used)):
                behavior_user_cumu_day1 = behavior_user_cumu[days_used[j].strftime("%Y-%m-%d")] 
                behavior_user_cumu_normal_day1 = behavior_user_cumu_normal[days_used[j].strftime("%Y-%m-%d")]
                behavior_user_raw_day1 = behavior_user_raw[days_used[j].strftime("%Y-%m-%d")]
                # drop time and reset the index the same
                behavior_user_cumu_day1 = behavior_user_cumu_day1.reset_index()
                behavior_user_cumu_normal_day1 = behavior_user_cumu_normal_day1.reset_index()
                behavior_user_raw_day1 = behavior_user_raw_day1.reset_index()
                behavior_user_cumu_day1 = behavior_user_cumu_day1[behavior_name + '_cumu'] 
                behavior_user_cumu_normal_day1 = behavior_user_cumu_normal_day1[behavior_name + '_cumu_normal']
                behavior_user_raw_day1 = behavior_user_raw_day1[behavior_name]
                df_graphs[days_used[j].strftime("%Y-%m-%d")] = behavior_user_cumu_day1
                df_graphs_normal[days_used[j].strftime("%Y-%m-%d")] = behavior_user_cumu_normal_day1
                df_graphs_raw[days_used[j].strftime("%Y-%m-%d")] = behavior_user_raw_day1
#                 df_graphs['Day ' + str(j+1)] = behavior_user_cumu_day1
#                 df_graphs_normal['Day ' + str(j+1)] = behavior_user_cumu_normal_day1
#                 df_graphs_raw['Day ' + str(j+1)] = behavior_user_raw_day1
                behavior_amount.append(behavior_user_cumu_day1.max())
            behavior_amount = pd.Series(behavior_amount)
            behavior_mean.append(behavior_amount.mean())
            behavior_std.append(behavior_amount.std())
            
            if len(df_graphs.columns) > 1:

                T10 = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 
                       'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
                line_styles = ['-', '--']
                
                # absolute    
                plt.figure()
                palette = plt.get_cmap('Set1')
                num=0
                line_styles_idx = 0
                for column in df_graphs.drop('x', axis=1):
                    if num >= len(T10):
                        num = 0
                    plt.plot(df_graphs['x'], df_graphs[column], marker='', color=T10[num], linewidth=1, 
                             alpha=0.9, label=column)
                    num+=1
                    line_styles_idx+=1
                plt.legend(loc=2, ncol=2)
                plt.title('cumulative duration')
                plt.xlabel("Time (hours)")

                plt.xticks(np.arange(0, 1441, step=180), ('0', '3', '6', '9', '12', '15', '18', '21', '24'))
                plt.ylabel(behavior_name +' duration')
                if not os.path.exists('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name):
                    os.makedirs('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name)
                plt.savefig('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED) + '/' + behavior_name + '/' + user + '_' + this_ema_day.strftime("%Y-%m-%d") + 
                            '_' + str(diff_mean.median()) + '.pdf', bbox_inches='tight')
                plt.close()

                
                # normal
                plt.figure()
                num=0
                line_styles_idx = 0
                for column in df_graphs_normal.drop('x', axis=1):
                    if num >= len(T10):
                        num = 0
                    plt.plot(df_graphs_normal['x'], df_graphs_normal[column], marker='', color=T10[num], 
                             linewidth=1, alpha=0.9, label=column)
                    num+=1
                    line_styles_idx+=1
                plt.legend(loc=2, ncol=2)
    #             plt.title('normalized cumulative duration')
                plt.xlabel("Time (hours)")
                plt.xticks(np.arange(0, 1441, step=180), ('0', '3', '6', '9', '12', '15', '18', '21', '24'))
                plt.ylabel('Normalized cumulative call duration')
#                 textstr = '\n'.join((
#                     'Stability Index: 0.976',
#                     'Percentile: 98/100'))
#                 plt.text(0, 0.8, textstr)
                if not os.path.exists('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name + '_normal'):
                    os.makedirs('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name + '_normal')
                plt.savefig('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED) + '/' + behavior_name + '_normal/' + user + '_' + this_ema_day.strftime("%Y-%m-%d") + 
                            '_normal_' + str(1.0-diff_normal_mean.median()) +'.pdf', bbox_inches='tight')
                plt.close()

                # raw    
                plt.figure()
                palette = plt.get_cmap('Set1')
                num=0
                line_styles_idx = 0
                for column in df_graphs_raw.drop('x', axis=1):
                    if num >= len(T10):
                        num = 0
                    plt.plot(df_graphs_raw['x'], df_graphs_raw[column], marker='', color=T10[num], 
                             linewidth=1, alpha=0.9, label=column)
                    num+=1
                    line_styles_idx+=1
                plt.legend(loc=2, ncol=2)
    #             plt.title('raw time series')
                plt.xlabel("Time (hours)")
                plt.xticks(np.arange(0, 1441, step=180), ('0', '3', '6', '9', '12', '15', '18', '21', '24'))
                plt.ylabel('Raw call duration')
#                 textstr = '\n'.join((
#                     'Absolute Instability Index: 1.172',
#                     'Percentile: 55/100')) 
#                 plt.text(0, 47.55, textstr)
                if not os.path.exists('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name + '_raw'):
                    os.makedirs('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED)+ '/' + behavior_name + '_raw')
                plt.savefig('graphs_ema_' + str(NUM_DAYS_USED) + '_' + str(NUM_DAYS_BLOCKED) + '/' + behavior_name + '_raw/' + user + '_' + this_ema_day.strftime("%Y-%m-%d") + 
                            '_raw_' + str(diff_raw_mean.median()) + '.pdf', bbox_inches='tight')
                plt.close()
            
        # put everything into the user's table       
        table_user[behavior_name +'_med_mean'] = absolute_cumu_med_mean_diff
        table_user[behavior_name +'_med_mean_normal'] = normal_cumu_med_mean_diff
        table_user[behavior_name +'_med_mean_raw'] = raw_med_mean_diff
        table_user[behavior_name + '_avg'] = behavior_mean
        table_user[behavior_name + '_std'] = behavior_std  
        table_user[behavior_name + '_num_good_days'] = num_good_days
#         table_user['ema_score'] = table_user['ema_neg_score'] - table_user['ema_pos_score']
        table_all_behav = pd.concat([table_all_behav,table_user])
        print('finished user: ' + user)
        print(datetime.datetime.now()) 
    return table_all_behav

In [7]:
behaviors = [
             calllog,
             smslog,
             smslog,
             unlock, 
             conver, 
             sleep, 
             act,
             act,
             act,
             act,
             act,
             act,
             light, 
             audio_amp,
             audio_infer,
             audio_infer,
             audio_infer
            ]
behavior_names = [
                  'calllog',
                  'sms_in',
                  'sms_out',
                  'unlock',
                  'conver',
                  'sleep',
                  'act_invehicle',
                  'act_onbike',
                  'act_onfoot',
                  'act_still',
                  'act_tilting',
                  'act_unknown',
                  'light',
                  'audio_amp',
                  'audio_noise',
                  'audio_silence',
                  'audio_voice'                  
]
print('number of behaviors:', len(behaviors))
print('number of behaviors:', len(behavior_names))
print()
      
print('the training window is:', TRAINING_WINDOW)
print('blocking previous ' + str(BLOCK_WINDOW) + ' days')

for i in range(len(behaviors)):
    behav_table = compute_stability_index(behaviors[i], behavior_names[i], TRAINING_WINDOW, BLOCK_WINDOW, 19)
    behav_table.to_csv(r'table_ema_' + str(TRAINING_WINDOW) + '_' + str(BLOCK_WINDOW) + '_'+ str(i) + '.csv')
    print('finished behavior ' + str(i))
    print()
    print()

In [8]:
def get_full_table(num_days_used, num_days_blocked):
    table0 = pd.read_csv('table_ema_'+str(num_days_used)+'_'+str(num_days_blocked)+'_0.csv')
    table0 =  table0.drop(['Unnamed: 0'], axis=1)
    for i in range(len(behavior_names)-1):
        new_table = pd.read_csv('table_ema_'+str(num_days_used)+'_'+str(num_days_blocked)+'_'+str(i+1)+'.csv')
        col_index = [c for c in new_table.columns if behavior_names[i+1] in c]
        table0 = pd.concat([table0, new_table[col_index]], axis=1)
    return table0

table0 = get_full_table(TRAINING_WINDOW,BLOCK_WINDOW)
table0.to_csv(r'table_ema_'+str(TRAINING_WINDOW)+'_'+str(BLOCK_WINDOW)+'.csv')